<a href="https://colab.research.google.com/github/zwelisizwe/homework-0/blob/master/Task_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

run('pip install --upgrade pip')

# Install apache-beam.
run('pip install --quiet apache-beam')


>> pip install --upgrade pip

>> pip install --quiet apache-beam



In [2]:
run('mkdir -p data')  # Create a directory if it doesn't exist
run('curl -o data/users_v.csv -L https://storage.googleapis.com/bdt-beam/users_v.csv')


>> mkdir -p data

>> curl -o data/users_v.csv -L https://storage.googleapis.com/bdt-beam/users_v.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  140k  100  140k    0     0   159k      0 --:--:-- --:--:-- --:--:--  159k



In [3]:
import apache_beam as beam

with beam.Pipeline() as pipeline:
    # Read the contents of the CSV file into a PCollection
    users_data = (
        pipeline
        | 'Read CSV File' >> beam.io.ReadFromText('data/users_v.csv')
    )


In [4]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = 'data/users_v.csv'
df = pd.read_csv(file_path)
df


,user_id,name,gender,age,address,date_joined
0,1,Anthony Wolf,male,73,New Rachelburgh-VA-49583,2019/03/13
1,2,James Armstrong,male,56,North Jillianfort-UT-86454,2020/11/06
2,3,Cody Shaw,male,75,North Anne-SC-53799,2004/05/29
3,4,Sierra Hamilton,female,76,New Angelafurt-ME-46190,2005/08/26
4,5,Chase Davis,male,31,South Bethmouth-WI-18562,2018/04/30
...,...,...,...,...,...,...
2352,2353,Brittney Graham,female,40,Brownland-CO-71229,2005/07/10
2353,2354,Allison Schmidt,female,43,Port Logan-MD-38588,2008/11/30
2354,2355,Christopher Johnson,male,68,North Justinton-VA-32798,2006/08/01
2355,2356,Mark Brown,male,67,New Kayleefurt-MA-82581,2013/11/16


In [6]:
import apache_beam as beam
from datetime import datetime

# Custom function to transform CSV rows into the desired format
def transform_csv_row(row):
    # Split the CSV row into individual elements
    elements = row.split(',')

    # Modify the elements as needed
    user_id, full_name, gender, age, location, date = elements

    # Parse the input date and format it as "YYYY-MM-DD"
    try:
        parsed_date = datetime.strptime(date, "%Y/%m/%d")
        formatted_date = parsed_date.strftime("%Y-%m-%d")
    except ValueError:
        formatted_date = date  # Keep the original date if parsing fails

    transformed_row = f"{user_id};{full_name.capitalize()};{gender.capitalize()};{age};{location.replace('-', ',')};{formatted_date}"

    return transformed_row

with beam.Pipeline() as pipeline:
    # Read the contents of the CSV file into a PCollection
    users_data = (
        pipeline
        | 'Read CSV File' >> beam.io.ReadFromText('data/users_v.csv')

        # Use beam.Map to transform each row into the desired format
        | 'Transform CSV Rows' >> beam.Map(transform_csv_row)
    )

    # Write the formatted data to a CSV file
    users_data | 'Write to CSV' >> beam.io.WriteToText('marketing_format.csv')


In [7]:
import apache_beam as beam
from datetime import datetime

# Custom function to transform CSV rows into key-value pairs (gender, 1)
def transform_csv_row(row):
    # Split the CSV row into individual elements
    elements = row.split(',')

    # Extract the gender field and capitalize it
    gender = elements[2].strip().capitalize()

    # Emit a key-value pair with the gender as the key and 1 as the value
    return (gender, 1)

with beam.Pipeline() as pipeline:
    # Read the contents of the CSV file into a PCollection
    users_data = (
        pipeline
        | 'Read CSV File' >> beam.io.ReadFromText('data/users_v.csv')

        # Use beam.Map to transform each row into key-value pairs
        | 'Transform CSV Rows' >> beam.Map(transform_csv_row)

        # Count the occurrences of each gender
        | 'Count Gender' >> beam.combiners.Count.PerKey()
    )

    # Print the results
    users_data | 'Print Results' >> beam.Map(print)


('Gender', 1)
('Male', 1207)
('Female', 1150)


In [ ]:
import apache_beam as beam

# Custom function to transform CSV rows into key-value pairs (join date, 1)
def transform_csv_row(row):
    # Split the CSV row into individual elements
    elements = row.split(',')

    # Extract the join date field
    join_date = elements[5].strip()

    # Emit a key-value pair with the join date as the key and 1 as the value
    return (join_date, 1)

with beam.Pipeline() as pipeline:
    # Read the contents of the CSV file into a PCollection
    users_data = (
        pipeline
        | 'Read CSV File' >> beam.io.ReadFromText('data/users_v.csv')

        # Use beam.Map to transform each row into key-value pairs
        | 'Transform CSV Rows' >> beam.Map(transform_csv_row)

        # Count the occurrences of each join date
        | 'Count Join Date' >> beam.combiners.Count.PerKey()
    )

    # Print the results
    users_data | 'Print Results' >> beam.Map(print)


In [ ]:
import apache_beam as beam

# Custom function to transform CSV rows into key-value pairs (state, 1)
def transform_csv_row(row):
    # Split the CSV row into individual elements
    elements = row.split(',')

    # Extract the state field
    address = elements[4].strip()

    # The state is the last part of the address, so we extract it
    state = address.split('-')[-1].strip()

    # Emit a key-value pair with the state as the key and 1 as the value
    return (state, 1)

with beam.Pipeline() as pipeline:
    # Read the contents of the CSV file into a PCollection
    users_data = (
        pipeline
        | 'Read CSV File' >> beam.io.ReadFromText('data/users_v.csv')

        # Use beam.Map to transform each row into key-value pairs
        | 'Transform CSV Rows' >> beam.Map(transform_csv_row)

        # Count the occurrences of each state
        | 'Count State' >> beam.combiners.Count.PerKey()
    )

    # Print the results
    users_data | 'Print Results' >> beam.Map(print)

